<a href="https://colab.research.google.com/github/Junghwan-brian/SDE-Net/blob/master/colab_sdenet_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Nov 18 12:30:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import datasets
import tensorflow as tf
import tensorflow_addons as tfa
from warnings import filterwarnings
import random
import math
from __future__ import print_function
import time
from scipy import misc
from numpy.linalg import inv
import os
import numpy as np
from tensorflow.keras.layers import Dense, ReLU, Layer
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential

filterwarnings("ignore")
def load_MSD():
# Load the raw data.
    num_attributes = 90
    names = ['Year'] + ['Attr_{}'.format(i) for i in range(num_attributes)]
    # YearPredictionMSD load file
    df = pd.read_csv('drive/MyDrive/SDE-Net/YearPredictionMSD.txt', header=None, names=names)

# Validate the data.
    num_examples = 515345
    assert len(df.columns) == num_attributes + 1
    assert len(df) == num_examples
    assert not df.isnull().values.any()


# Train/test split. See "Data Set Information".
    num_train = 463715
    df = df.values
    train = df[:num_train]
    test = df[num_train:]
    del df


# Seperate inputs and outputs.
    X_train, y_train = train[:, 1:], train[:, 0]
    X_test, y_test = test[:, 1:], test[:, 0]
    del train
    del test
    
    standardize = StandardScaler().fit(X_train)
    X_train = standardize.transform(X_train)
    X_test = standardize.transform(X_test)

    y_train1 = np.expand_dims(y_train, axis=1)
    y_test1 = np.expand_dims(y_test , axis=1)

    standardize2 = StandardScaler().fit(y_train1)
    y_train = standardize2.transform(y_train1)
    y_train = np.squeeze(y_train)

    y_test1 = np.expand_dims(y_test, axis=1)
    y_test = standardize2.transform(y_test1)
    y_test = np.squeeze(y_test)
    
    return X_train, y_train, X_test, y_test


def load_boston():
    boston = datasets.load_boston()
    x = boston.data
    X = np.concatenate((x,x), axis=1)
    for i in range(5):
        X = np.concatenate((X,x), axis=1)
    X = X[:,0:-1]
    standardize = StandardScaler().fit(X)
    X = standardize.transform(X)
    return X



def load_dataset(dataset):
    if dataset == 'MSD':
        X_train, y_train, X_test, y_test = load_MSD()
        return X_train, y_train, X_test, y_test
    if dataset == 'boston':
        x = load_boston()
        return x
    
    

In [3]:
tf.keras.backend.set_floatx("float64")
tf.random.set_seed(0)


class Drift(Layer):
    def __init__(self):
        super(Drift, self).__init__(name="drift_net")
        self.fc = Dense(50, kernel_initializer="he_normal",activation = 'relu')  # input : 50

    def call(self, t, x):
        out = self.fc(x)
        return out


class Diffusion(Layer):
    def __init__(self):
        super(Diffusion, self).__init__(name="diffusion_net")
        self.fc1 = Dense(100, kernel_initializer="he_normal",activation = 'relu')  # input : 50
        self.fc2 = Dense(1)  # input : 100

    def call(self, t, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = tf.nn.sigmoid(out)
        return out  # batch,1


class SDENet(Model):
    def __init__(self, layer_depth):
        super(SDENet, self).__init__(name="SDE_Net")
        self.layer_depth = layer_depth
        self.downsampling_layers = Dense(
            50, kernel_initializer="he_normal"
        )  # batch, 50
        self.drift = Drift()  # batch, 1
        self.diffusion = Diffusion()
        self.fc_layers = Sequential(
            [ReLU(), Dense(2)]
        )  # input : 50, output : mean, variance
        self.deltat = 4.0 / self.layer_depth  # T:4, N:layer_depth
        self.sigma_max = 0.5  # sigma_max : scaling diffusion output

    def call(self, x, training_diffusion=False):
        out = self.downsampling_layers(x)
        if not training_diffusion:
            t = 0
            diffusion_term = self.sigma_max * self.diffusion(t, out)
            for i in range(self.layer_depth):
                t = 4 * (float(i)) / self.layer_depth
                out = (
                    out
                    + self.drift(t, out) * self.deltat
                    + diffusion_term
                    * tf.cast(tf.math.sqrt(self.deltat), "float64")
                    * tf.random.normal(tf.shape(out), dtype="float64")
                )  # Euler-Maruyama method

            final_out = self.fc_layers(out)
            mean = final_out[:, 0]
            # sigma should be greater than 0.
            sigma = tf.math.softplus(final_out[:, 1]) + 1e-3
            return mean, sigma

        else:
            t = 0
            final_out = self.diffusion(t, out)
            return final_out


In [5]:

filterwarnings("ignore")
epochs = 1
lr = 1e-4
lr2 = 0.01
seed = 0
batch_size = 128
target_scale = 10.939756
# Data
print("==> Preparing data..")

tf.random.set_seed(seed)

X_train, y_train, X_test, y_test = load_dataset("MSD")

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
)

# Model
print("==> Building model..")
model = SDENet(4)

real_label = 0  # training data
fake_label = 1  # out-of-distribution data

criterion = tf.keras.losses.BinaryCrossentropy()


# optimize drift net
optimizer_drift = tfa.optimizers.SGDW(learning_rate=lr, weight_decay=5e-4,)

# optimize fc layer
optimizer_fc = tfa.optimizers.SGDW(learning_rate=lr, weight_decay=5e-4,)

# optimize down sampling layer
optimizer_dsl = tfa.optimizers.SGDW(learning_rate=lr, weight_decay=5e-4,)

# optimize diffusion net
optimizer_diffusion = tfa.optimizers.SGDW(
    learning_rate=lr2, weight_decay=5e-4,
)


def nll_loss(y, mean, sigma):
    loss = tf.math.reduce_mean(tf.math.log(sigma ** 2) + (y - mean) ** 2 / (sigma ** 2))
    return loss


mse = tf.keras.losses.MSE

train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()
in_loss = tf.keras.metrics.Mean()
out_loss = tf.keras.metrics.Mean()


@tf.function
def train_net(x, y):
    with tf.GradientTape(persistent=True) as tape:
        mean, sigma = model(x, training_diffusion=False)
        loss = nll_loss(y, mean, sigma)

    drift_gradient = tape.gradient(loss, model.drift.trainable_variables)
    dsl_gradient = tape.gradient(loss, model.downsampling_layers.trainable_variables)
    fc_gradient = tape.gradient(loss, model.fc_layers.trainable_variables)

    drift_gradient = [(tf.clip_by_norm(grad, 100)) for grad in drift_gradient]
    dsl_gradient = [(tf.clip_by_norm(grad, 100)) for grad in dsl_gradient]
    fc_gradient = [(tf.clip_by_norm(grad, 100)) for grad in fc_gradient]

    optimizer_drift.apply_gradients(
        zip(drift_gradient, model.drift.trainable_variables)
    )
    optimizer_dsl.apply_gradients(
        zip(dsl_gradient, model.downsampling_layers.trainable_variables)
    )
    optimizer_fc.apply_gradients(zip(fc_gradient, model.fc_layers.trainable_variables))
    train_loss(loss)


@tf.function
def train_diffusion(real_x):
    with tf.GradientTape(watch_accessed_variables=False) as real_tape_diffusion:
        # only access to diffusion layer's parameters
        real_tape_diffusion.watch(model.diffusion.trainable_variables)
        real_y = tf.fill((real_x.shape[0], 1), real_label)
        real_pred = model(real_x, training_diffusion=True)
        real_loss = mse(real_y, real_pred)

    diffusion_gradient = real_tape_diffusion.gradient(
        real_loss, model.diffusion.trainable_variables
    )

    diffusion_gradient1 = [(tf.clip_by_norm(grad, 100)) for grad in diffusion_gradient]

    with tf.GradientTape(watch_accessed_variables=False) as fake_tape_diffusion:
        fake_tape_diffusion.watch(model.diffusion.trainable_variables)
        # fake std is 2 in official code, but in paper it is 4
        fake_x = (
            tf.cast(
                tf.random.normal((real_x.shape[0], 90), mean=0, stddev=2), "float64"
            )
            + real_x
        )
        fake_y = tf.fill((real_x.shape[0], 1), fake_label)
        fake_pred = model(fake_x, training_diffusion=True)
        fake_loss = mse(fake_y, fake_pred)

    diffusion_gradient = fake_tape_diffusion.gradient(
        fake_loss, model.diffusion.trainable_variables
    )

    diffusion_gradient2 = [(tf.clip_by_norm(grad, 100)) for grad in diffusion_gradient]

    diffusion_gradient = [
        (grad1 + grad2) for grad1, grad2 in zip(diffusion_gradient1, diffusion_gradient2)
    ]

    optimizer_diffusion.apply_gradients(
        zip(diffusion_gradient, model.diffusion.trainable_variables)
    )
    in_loss(real_loss)
    out_loss(fake_loss)


@tf.function
def test_net(x, y):
    current_mean = 0
    for i in range(10):
        mean, sigma = model(x, training_diffusion=False)
        current_mean += mean
    current_mean = current_mean / 10
    loss = mse(y, current_mean) * target_scale

    test_loss(loss)

# Training
def train():

    for data, label in train_ds:
        train_net(data, label)
        train_diffusion(data)

    print(
        f"Loss: {train_loss.result():.4f}, Loss_in: {in_loss.result():.4f}, Loss_out: {out_loss.result():.4f}"
    )
    train_loss.reset_states()
    in_loss.reset_states()
    out_loss.reset_states()


# Testing
def test():
    for data, label in test_ds:
        test_net(data, label)
    print(f"Test Loss : {test_loss.result():.4f}")
    test_loss.reset_states()


for epoch in range(epochs):
    print(f"\nEpoch: {epoch + 1}")

    if epoch == 0:
        # In the thesis, initial sigma_max is 0.01 but in the author's code it is 0.1
        model.sigma_max = 0.1
    if epoch == 30:
        model.sigma_max = 0.5
    train()
    test()

path = "sde_net.h5"
tf.saved_model.save(model, path)


==> Preparing data..
==> Building model..

Epoch: 1

If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Loss: 78975.9072, Loss_in: 0.0195, Loss_out: 0.0126
Test Loss : 10.6518
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: sde_net.h5/assets


In [6]:
# Training settings
eval_iter = 10
seed = 0
gpu = 0
outf = "test_sde"
batch_size = 512
target_scale = 10.939756

tf.random.set_seed(seed)



in_test_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
)


X_out = load_dataset("boston")

os.makedirs(outf,exist_ok=True)
# then make file : confidence_Base_In.txt, confidence_Base_Out.txt


def tpr95(dir_name):
    # calculate the falsepositive error when tpr is 95%
    cifar = np.loadtxt("%s/confidence_Base_In.txt" % dir_name, delimiter=",")
    other = np.loadtxt("%s/confidence_Base_Out.txt" % dir_name, delimiter=",")

    Y1 = other
    X1 = cifar
    end = np.max([np.max(X1), np.max(Y1)])
    start = np.min([np.min(X1), np.min(Y1)])
    gap = (end - start) / 200000  # precision:200000

    total = 0.0
    fpr = 0.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        if tpr <= 0.96 and tpr >= 0.94:
            fpr += error2
            total += 1
    if total == 0:
        print("corner case")
        fprBase = 1
    else:
        fprBase = fpr / total

    return fprBase


def auroc(dir_name):
    # calculate the AUROC
    f1 = open("%s/Update_Base_ROC_tpr.txt" % dir_name, "w")
    f2 = open("%s/Update_Base_ROC_fpr.txt" % dir_name, "w")
    cifar = np.loadtxt("%s/confidence_Base_In.txt" % dir_name, delimiter=",")
    other = np.loadtxt("%s/confidence_Base_Out.txt" % dir_name, delimiter=",")

    Y1 = other
    X1 = cifar
    end = np.max([np.max(X1), np.max(Y1)])
    start = np.min([np.min(X1), np.min(Y1)])
    gap = (end - start) / 200000

    aurocBase = 0.0
    fprTemp = 1.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
        fpr = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        f1.write("{}\n".format(tpr))
        f2.write("{}\n".format(fpr))
        aurocBase += (-fpr + fprTemp) * tpr
        fprTemp = fpr
    f1.close()
    f2.close()
    return aurocBase


def auprIn(dir_name):
    # calculate the AUPR
    cifar = np.loadtxt("%s/confidence_Base_In.txt" % dir_name, delimiter=",")
    other = np.loadtxt("%s/confidence_Base_Out.txt" % dir_name, delimiter=",")

    precisionVec = []
    recallVec = []
    Y1 = other
    X1 = cifar
    end = np.max([np.max(X1), np.max(Y1)])
    start = np.min([np.min(X1), np.min(Y1)])
    gap = (end - start) / 200000

    auprBase = 0.0
    recallTemp = 1.0
    for delta in np.arange(start, end, gap):
        tp = np.sum(np.sum(X1 >= delta))  # / np.float(len(X1))
        fp = np.sum(np.sum(Y1 >= delta))  # / np.float(len(Y1))
        if tp + fp == 0:
            continue
        precision = tp / (tp + fp)
        recall = tp / np.float(len(X1))
        precisionVec.append(precision)
        recallVec.append(recall)
        auprBase += (recallTemp - recall) * precision
        recallTemp = recall
    auprBase += recall * precision

    return auprBase


def auprOut(dir_name):
    # calculate the AUPR
    cifar = np.loadtxt("%s/confidence_Base_In.txt" % dir_name, delimiter=",")
    other = np.loadtxt("%s/confidence_Base_Out.txt" % dir_name, delimiter=",")
    Y1 = other
    X1 = cifar
    end = np.max([np.max(X1), np.max(Y1)])
    start = np.min([np.min(X1), np.min(Y1)])
    gap = (end - start) / 200000

    auprBase = 0.0
    recallTemp = 1.0
    for delta in np.arange(end, start, -gap):
        fp = np.sum(np.sum(X1 < delta))  # / np.float(len(X1))
        tp = np.sum(np.sum(Y1 < delta))  # / np.float(len(Y1))
        if tp + fp == 0:
            break
        precision = tp / (tp + fp)
        recall = tp / np.float(len(Y1))
        auprBase += (recallTemp - recall) * precision
        recallTemp = recall
    auprBase += recall * precision

    return auprBase


def detection(dir_name):
    # calculate the minimum detection error
    cifar = np.loadtxt("%s/confidence_Base_In.txt" % dir_name, delimiter=",")
    other = np.loadtxt("%s/confidence_Base_Out.txt" % dir_name, delimiter=",")

    Y1 = other
    X1 = cifar
    end = np.max([np.max(X1), np.max(Y1)])
    start = np.min([np.min(X1), np.min(Y1)])
    gap = (end - start) / 200000

    errorBase = 1.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        errorBase = np.minimum(errorBase, (tpr + error2) / 2.0)

    return errorBase


def metric(dir_name, task="OOD"):
    print("{}{:>34}".format(task, "Performance of Baseline detector"))
    fprBase = tpr95(dir_name)
    print("{:20}{:13.3f}%".format("TNR at TPR 95%:", (1 - fprBase) * 100))
    aurocBase = auroc(dir_name)
    print("{:20}{:13.3f}%".format("AUROC:", aurocBase * 100))
    errorBase = detection(dir_name)
    print("{:20}{:13.3f}%".format("Detection acc:", (1 - errorBase) * 100))
    auprinBase = auprIn(dir_name)
    print("{:20}{:13.3f}%".format("AUPR In:", auprinBase * 100))
    auproutBase = auprOut(dir_name)
    print("{:20}{:13.3f}%".format("AUPR Out:", auproutBase * 100))


In [8]:

Iter_test = 100

def mse(y, mean):
    loss = tf.math.reduce_mean((y - mean) ** 2)
    return loss


def generate_target():
    f1 = open("%s/confidence_Base_In.txt" % outf, "w")
    test_loss = 0
    for data, targets in in_test_ds:
        current_mean = 0
        for j in range(eval_iter):
            mean, sigma = model(data)
            current_mean = mean + current_mean
            if j == 0:
                Sigma = tf.expand_dims(sigma, 1)
                Mean = tf.expand_dims(mean, 1)
            else:
                Sigma = tf.concat((Sigma, tf.expand_dims(sigma, 1)), axis=1)
                Mean = tf.concat((Mean, tf.expand_dims(mean, 1)), axis=1)
        current_mean = current_mean / eval_iter
        loss = mse(targets, current_mean)
        test_loss += loss.numpy()
        Var_mean = tf.math.reduce_std(Mean,axis=-1)
        for i in range(data.shape[0]):
            soft_out = Var_mean[i].numpy()
            f1.write("{}\n".format(-soft_out))

    f1.close()

    print("\n Final RMSE: {}".format(np.sqrt(test_loss / Iter_test) * target_scale))


def generate_non_target():
    f2 = open("%s/confidence_Base_Out.txt" % outf, "w")
    current_mean = 0
    for j in range(eval_iter):
        mean, sigma = model(X_out)
        current_mean = mean + current_mean
        if j == 0:
            Sigma = tf.expand_dims(sigma, 1)
            Mean = tf.expand_dims(mean, 1)
        else:
            Sigma = tf.concat((Sigma, tf.expand_dims(sigma, 1)), axis=1)
            Mean = tf.concat((Mean, tf.expand_dims(mean, 1)), axis=1)

    Var_mean = tf.math.reduce_std(Mean,axis=-1)
    for i in range(X_out.shape[0]):
        soft_out = Var_mean[i].numpy()
        f2.write("{}\n".format(-soft_out))
    f2.close()


print("generate log from in-distribution data")
generate_target()
print("generate log  from out-of-distribution data")
generate_non_target()
print("calculate metrics for OOD")
metric(outf, "OOD")


generate log from in-distribution data

 Final RMSE: 32.49121250125466
generate log  from out-of-distribution data
calculate metrics for OOD
OOD  Performance of Baseline detector
TNR at TPR 95%:            40.091%
AUROC:                     78.295%
Detection acc:             73.974%
AUPR In:                   99.956%
AUPR Out:                   0.717%
